In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

plt.style.use('seaborn-ticks')
%matplotlib inline

In [2]:
# Wczytanie danych
df = pd.read_csv("data/ver3.csv")
print(df.shape)
pd.set_option("display.max_columns",50)
df.head()

(3603, 11)


,male,age,education,currentSmoker,prevalentHyp,heartRate,TenYearCHD,log_glucose,log_BMI,log_sysBP,log_totChol
0,1,39,4.0,0,0,80.0,0,4.343805,3.294725,4.663439,5.273000
1,0,46,2.0,0,0,95.0,0,4.330733,3.357942,4.795791,5.521461
2,1,48,1.0,1,0,75.0,0,4.248495,3.232384,4.848116,5.501258
3,0,61,3.0,1,1,65.0,1,4.634729,3.352707,5.010635,5.416100
4,0,46,3.0,1,0,85.0,0,4.442651,3.139833,4.867534,5.652489


In [3]:
features = df.columns.tolist()
features.remove('TenYearCHD')
rhs = "+".join(features)

In [4]:
# Zaimportowanie funkcji train_test_split z sklearn
from sklearn.model_selection import train_test_split
# Podzielenie zbioru na treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(df, df.TenYearCHD, test_size=0.3, random_state=0)
print(X_train.shape, X_test.shape)

(2522, 11) (1081, 11)


In [5]:
for k in range(1, 10):
    X_train, X_test, y_train, y_test = train_test_split(df, df.TenYearCHD, test_size=0.1*k, random_state=2020)
    mod = sm.GLM.from_formula(formula="TenYearCHD~"+rhs, data=X_train, 
                              family=sm.families.Binomial())
    res = mod.fit()
    # Liczymy predykcje na zbiorze treningowym
    predsTrain = res.predict()
    # Liczymy predykcje na zbiorze walidacyjnym
    preds = res.predict(X_test)
    print(f"test: {k/10}, Train AUC:", round(roc_auc_score(y_train, predsTrain), 4),
          "Valid AUC:", round(roc_auc_score(y_test, preds), 4))

test: 0.1, Train AUC: 0.7314 Valid AUC: 0.7163
test: 0.2, Train AUC: 0.7329 Valid AUC: 0.7192
test: 0.3, Train AUC: 0.732 Valid AUC: 0.7263
test: 0.4, Train AUC: 0.7264 Valid AUC: 0.7337
test: 0.5, Train AUC: 0.725 Valid AUC: 0.7308
test: 0.6, Train AUC: 0.7142 Valid AUC: 0.735
test: 0.7, Train AUC: 0.7256 Valid AUC: 0.7267
test: 0.8, Train AUC: 0.736 Valid AUC: 0.7123
test: 0.9, Train AUC: 0.7526 Valid AUC: 0.6893


Przetrenowanie powyżej 0.7, dobry wynik dla 0.3.

In [6]:
# Wczytanie funkcji KFold
from sklearn.model_selection import KFold

# Stworzenie funkcji do dzielenia foldów (w tym przypadku w walidacji 10 razy składanej)
kf = KFold(n_splits=10, shuffle=True, random_state=2020)

# Aby oszczędzać pamięć informacja o foldach to wyłącznie numery wierszy
for train, test in kf.split(df.index.values): # tutaj następuje podział
    # Stworzenie i estymacja modelu
    mod = sm.GLM.from_formula(formula="TenYearCHD~"+rhs,
                              data=df.iloc[train], family=sm.families.Binomial())
    res = mod.fit()
    # Zapisanie predykcji na zbiorze treningowym w wektorze predsTrain
    predsTrain = res.predict()
    # Zapisanie predykcji na zbiorze walidacyjnym w wektorze predsTest
    predsTest = res.predict(df.iloc[test])
    print("Train AUC:", np.round(roc_auc_score(df.TenYearCHD.iloc[train], predsTrain), 4),
          "Valid AUC:", np.round(roc_auc_score(df.TenYearCHD.iloc[test], predsTest), 4))

Train AUC: 0.7314 Valid AUC: 0.7163
Train AUC: 0.7319 Valid AUC: 0.7104
Train AUC: 0.7301 Valid AUC: 0.734
Train AUC: 0.7262 Valid AUC: 0.7614
Train AUC: 0.7305 Valid AUC: 0.7225
Train AUC: 0.7261 Valid AUC: 0.7633
Train AUC: 0.7367 Valid AUC: 0.668
Train AUC: 0.7318 Valid AUC: 0.7189
Train AUC: 0.7314 Valid AUC: 0.7186
Train AUC: 0.7317 Valid AUC: 0.7178


In [7]:
def CVTest(nFolds = 5, randomState=2020, debug=False):
    kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)

    # Listy do zapisywania wyników
    testResults = []
    trainResults = []
    predictions = []
    indices = []
    
    for train, test in kf.split(df.index.values):
        # Estymacja modelu GLM
        mod = sm.GLM.from_formula(formula="TenYearCHD~"+rhs,
                                  data=df.iloc[train], family=sm.families.Binomial())
        res = mod.fit()
        predsTrain = res.predict()
        preds = res.predict(df.iloc[test])
        
        # Zachowajmy informacje o predykcjach dla tego foldu
        predictions.append(preds.tolist().copy())
        
        # Razem z indeksami w oryginalnym data frame
        indices.append(df.iloc[test].index.tolist().copy())
        
        # Informowanie o każdym foldzie razem z wynikami treningowymi możemy opcjonalnie wyświetlać w trakcie
        trainScore = roc_auc_score((df.TenYearCHD.iloc[train]==1).astype(int), predsTrain)
        testScore = roc_auc_score((df.TenYearCHD.iloc[test]==1).astype(int), preds)
        
        # Zapisanie wyników dopasowania w foldach
        trainResults.append(trainScore)
        testResults.append(testScore)
        
        if debug:
            print("Train AUC:", trainScore,
                  "Valid AUC:", testScore)

    return trainResults, testResults, predictions, indices
 

In [8]:
trainResults, testResults, predictions, indices = CVTest(nFolds = 10, randomState=2020)
print(np.mean(trainResults), np.mean(testResults))

0.7307738206922597 0.7231204468226855


In [9]:
resultM1 = {
    "name": "Ekonometria",
    "description":"Model ekonometryczny",
    "specification": "TenYearCHD~"+rhs,
    "trainResults":trainResults.copy(),
    "testResults":testResults.copy(),
    "predictions":predictions.copy(),
    "indices":indices.copy(),
}

In [10]:
import pickle

# Otwieramy plik do zapisu binarnego z wykorzystenim with
with open("model_ekonometria_1_ver3.p", "wb") as fp:
    # Zapisujemy obiekt do wskaźnika pliku
    pickle.dump(resultM1, fp)